# Exploratory Data Analysis of Heavy Truck J1939 data

Ce notebook va servir de base dans l'analyse des données du `Heavy Truck`. Ces données reprennent une payload que nous pouvons retrouver avec le protocole **bus CAN J1939**.

**Payload d'un message sous protocole J1939:**

![payload j1939](./images/j1939-payload.png)

:warning: Attention à bien écouter Vendredi 26 la présentation de `Simon Bellemare` sur les données **bus CAN J1939** !

## Imports

In [1]:
from sqlmodel import Session, select

from canlock.db.database import get_session, init_db
from canlock.db.models import Vehicle, CanMessage, PgnDefinition, SpnDefinition, Ecu

In [2]:
init_db()  # Ensure tables are created

with get_session() as session:
    # Fetch all vehicles
    vehicles = session.exec(select(Vehicle)).all()
    
    # Fetch 100 CAN messages for example
    canmessages = session.exec(select(CanMessage).limit(100)).all()

In [3]:
vehicles

[Vehicle(id=UUID('206734e6-4f14-4da3-a448-0d460660ac78'), make='Renault', license_plate='Unknown', model='T520', year=2021)]

In [4]:
canmessages

[CanMessage(session_id=UUID('654d877b-afab-4119-9dac-5eb612928099'), timestamp=datetime.datetime(2020, 11, 23, 8, 3, 31, 985194), payload=b'\xff\xff\xff\xff\xff\xff\xff\xff', can_identifier=217056230, length=8, id=1),
 CanMessage(session_id=UUID('654d877b-afab-4119-9dac-5eb612928099'), timestamp=datetime.datetime(2020, 11, 23, 8, 3, 31, 986173), payload=b'\xff\xff\xff/\xff\xff\xff\xff', can_identifier=285180134, length=8, id=2),
 CanMessage(session_id=UUID('654d877b-afab-4119-9dac-5eb612928099'), timestamp=datetime.datetime(2020, 11, 23, 8, 3, 31, 986899), payload=b'\x00\x00\x00\x00\x00\x00\x00\x00', can_identifier=369056230, length=8, id=3),
 CanMessage(session_id=UUID('654d877b-afab-4119-9dac-5eb612928099'), timestamp=datetime.datetime(2020, 11, 23, 8, 3, 31, 986988), payload=b'\xff\xff\xff\xff\xff\xff\xff\xff', can_identifier=418383590, length=8, id=4),
 CanMessage(session_id=UUID('654d877b-afab-4119-9dac-5eb612928099'), timestamp=datetime.datetime(2020, 11, 23, 8, 3, 31, 987066), p

In [5]:
canmessages[0].can_identifier

# Convert integer to 29 bits string
binary_identifier = bin(canmessages[0].can_identifier)[2:].zfill(29)
pgn_identifier = binary_identifier[3:21] # Extract bits 3 to 20

# Convert PGN identifier to integer
pgn_integer = int(pgn_identifier, 2)

# SQLmodel query to table PGNDefinition with pgn_integer key value
with get_session() as session:
    pgn_definition = session.exec(select(PgnDefinition).where(PgnDefinition.pgn_identifier == pgn_integer)).first()

In [6]:
pgn_identifier

'001111000000000011'

In [7]:
pgn_integer

61443

In [8]:
pgn_definition.name

'ELECTRONIC_ENGINE_CONTROLLER_2'

In [9]:
canmessages[0].can_identifier

# Convert integer to 29 bits string
binary_identifier = bin(canmessages[0].can_identifier)[2:].zfill(29)
address_identifier = binary_identifier[21:] # Extract bits 21 to the end

# Convert Address identifier to integer
address_integer = int(address_identifier, 2)

# SQLmodel query to table Ecu with pgn_integer key value
with get_session() as session:
    ecu_result = session.exec(select(Ecu).where(Ecu.address == address_integer)).first()

In [10]:
ecu_result.name

'Body-to-Vehicle Interface Control'

In [11]:
with get_session() as session:
    all_spn_definitions = session.exec(select(SpnDefinition)).all()
    all_pgn_definitions = session.exec(select(PgnDefinition)).all()

In [12]:
len(all_spn_definitions)

13313

In [13]:
len(all_pgn_definitions)

2520

In [14]:
all_spn_names = [spn_def.name for spn_def in all_spn_definitions]
all_pgn_names = [pgn_def.name for pgn_def in all_pgn_definitions]

In [15]:
set(all_spn_names).intersection(set(all_pgn_names))

{'ASSEMBLY_UNIT_IDENTIFICATION',
 'AXLE_GROUP_WEIGHT',
 'ECONOMY_MODE_ESTIMATED_INSTANTANEOUS_FUEL_SAVINGS_RATE',
 'ENGINE_AIR_START_PRESSURE',
 'ENGINE_EXHAUST_NOX',
 'GROSS_COMBINATION_VEHICLE_WEIGHT',
 'PAYLOAD_TEMPERATURE_1',
 'PAYLOAD_TEMPERATURE_2',
 'REQUESTED_GENERATOR_TOTAL_AC_REACTIVE_POWER',
 'SOFTWARE_IDENTIFICATION',
 'ZERO_NET_VEHICLE_WEIGHT_CHANGE'}

In [16]:
all_spn_definitions

[SpnDefinition(id=UUID('b95e68ae-df8e-467e-8c01-952e06cf2a8f'), is_analog=True, bit_length=2, analog_attributes_id=UUID('3dcbe7a2-fa1f-4205-aee1-49a450b56a2d'), spn_identifier=695, name='ENGINE_OVERRIDE_CONTROL_MODE', bit_start=0, pgn_id=UUID('790b2432-4182-4351-88d5-d9066e7d436a')),
 SpnDefinition(id=UUID('ab2aec7d-6eee-4865-847e-75d277dd3cde'), is_analog=True, bit_length=2, analog_attributes_id=UUID('504098bd-3b4a-4b98-a608-7a92458ea615'), spn_identifier=696, name='ENGINE_REQUESTED_SPEED_CONTROL_CONDITIONS', bit_start=2, pgn_id=UUID('790b2432-4182-4351-88d5-d9066e7d436a')),
 SpnDefinition(id=UUID('66554776-5582-414e-a8a0-4d6af9e26785'), is_analog=True, bit_length=2, analog_attributes_id=UUID('c4a781f7-911e-4e3c-b8f6-a332eb53ec28'), spn_identifier=897, name='OVERRIDE_CONTROL_MODE_PRIORITY', bit_start=4, pgn_id=UUID('790b2432-4182-4351-88d5-d9066e7d436a')),
 SpnDefinition(id=UUID('9e9854de-df01-4632-b713-59d391cd6ef3'), is_analog=True, bit_length=16, analog_attributes_id=UUID('35e6f370

In [17]:
all_pgn_definitions

[PgnDefinition(id=UUID('790b2432-4182-4351-88d5-d9066e7d436a'), vehicle_id=UUID('206734e6-4f14-4da3-a448-0d460660ac78'), pgn_identifier=0, name='TORQUE_SPEED_CONTROL_1'),
 PgnDefinition(id=UUID('558de880-69b2-42e2-b051-87d64cbbb9db'), vehicle_id=UUID('206734e6-4f14-4da3-a448-0d460660ac78'), pgn_identifier=256, name='TRANSMISSION_CONTROL_1'),
 PgnDefinition(id=UUID('cd4a902f-f52a-4729-9921-6d772be3a396'), vehicle_id=UUID('206734e6-4f14-4da3-a448-0d460660ac78'), pgn_identifier=1024, name='EXTERNAL_BRAKE_REQUEST'),
 PgnDefinition(id=UUID('349f3372-6e63-409d-bd2b-d5961061530d'), vehicle_id=UUID('206734e6-4f14-4da3-a448-0d460660ac78'), pgn_identifier=1792, name='GENERAL_PURPOSE_VALVE_PRESSURE'),
 PgnDefinition(id=UUID('39f16662-edd1-42f6-a139-be597d4b4b2c'), vehicle_id=UUID('206734e6-4f14-4da3-a448-0d460660ac78'), pgn_identifier=2048, name='AUXILIARY_INPUT_OUTPUT_STATUS_5'),
 PgnDefinition(id=UUID('5e80146e-2f80-4439-a612-5eebcd32fe09'), vehicle_id=UUID('206734e6-4f14-4da3-a448-0d460660ac78

In [18]:
# Convert payload to 64-bits binary string
binary_payload = bin(int.from_bytes(canmessages[7].payload, byteorder='big'))[2:].zfill(64)
binary_payload

'0000000000000000000000000000000010101110000111110000000010000000'

In [19]:
binary_payload[4:12]

'00000000'

In [20]:
binary_payload[24:40]

'0000000010101110'

In [21]:
int(binary_payload[24:40], 2)

174

In [25]:
with get_session() as session:
    spn_definition = session.exec(select(SpnDefinition).where(SpnDefinition.spn_identifier == int(binary_payload[24:40], 2))).first()
    analog_definition = spn_definition.analog_attributes

In [26]:
analog_definition

AnalogAttributes(is_big_endian=False, is_signed=False, offset=-40.0, id=UUID('d413016e-7d85-479d-9c85-d4b9bb78e13c'), scale=1.0, unit='°C')

In [27]:
binary_payload[8:16]

'00000000'